In [84]:
import cv2
import numpy as np
import os
import scipy.io as sio
from cvprlab_python.lab3.extract_random import extract_random
from cvprlab_python.lab3.cvpr_compare import cvpr_compare
from random import randint
from matplotlib import pyplot as plt


In [85]:
def extract_color_descriptor(img):
    # Compute the average red, green, and blue values as a basic color descriptor
    R = np.mean(img[:, :, 2])  # Note: OpenCV uses BGR format
    G = np.mean(img[:, :, 1])
    B = np.mean(img[:, :, 0])
    return np.array([R, G, B])

In [86]:
DATASET_FOLDER = "cvprlab_python/MSRC_ObjCategImageDatabase_v2"
OUT_FOLDER = "cvprlab_python/lab3/descriptors"
OUT_SUBFOLDER = 'globalRGBhisto'

if not os.path.exists(OUT_FOLDER):
    os.makedirs(OUT_FOLDER)

# Extract Image Descriptors from the Dataset
os.makedirs(os.path.join(OUT_FOLDER, OUT_SUBFOLDER), exist_ok=True)

In [87]:
for filename in os.listdir(os.path.join(DATASET_FOLDER, 'Images')):
    if filename.endswith(".bmp"):
        print(f"Processing file {filename}")
        img_path = os.path.join(DATASET_FOLDER, 'Images', filename)
        img = cv2.imread(img_path).astype(np.float64) / 255.0  # Normalize the image
        fout = os.path.join(OUT_FOLDER, OUT_SUBFOLDER, filename.replace('.bmp', '.mat'))
        
        # Call extractRandom (or another feature extraction function) to get the descriptor
        F = extract_random(img)        

        # Save the descriptor to a .mat file
        sio.savemat(fout, {'F': F})

Processing file 17_15_s.bmp
Processing file 16_15_s.bmp
Processing file 14_27_s.bmp
Processing file 3_2_s.bmp
Processing file 1_20_s.bmp
Processing file 8_5_s.bmp
Processing file 2_12_s.bmp
Processing file 3_12_s.bmp
Processing file 18_20_s.bmp
Processing file 19_20_s.bmp
Processing file 13_3_s.bmp
Processing file 18_4_s.bmp
Processing file 10_2_s.bmp
Processing file 20_18_s.bmp
Processing file 17_28_s.bmp
Processing file 16_28_s.bmp
Processing file 8_10_s.bmp
Processing file 9_10_s.bmp
Processing file 7_18_s.bmp
Processing file 1_7_s.bmp
Processing file 9_1_s.bmp
Processing file 6_18_s.bmp
Processing file 2_6_s.bmp
Processing file 11_6_s.bmp
Processing file 5_17_s.bmp
Processing file 4_17_s.bmp
Processing file 12_7_s.bmp
Processing file 7_25_s.bmp
Processing file 6_25_s.bmp
Processing file 12_22_s.bmp
Processing file 13_22_s.bmp
Processing file 10_10_s.bmp
Processing file 11_10_s.bmp
Processing file 7_27_s.bmp
Processing file 6_27_s.bmp
Processing file 12_5_s.bmp
Processing file 5_15_

In [91]:
DESCRIPTOR_FOLDER = "cvprlab_python/lab3/descriptors"
DESCRIPTOR_SUBFOLDER = 'globalRGBhisto'

# Load all descriptors
ALLFEAT = []
ALLFILES = []
for filename in os.listdir(os.path.join(DESCRIPTOR_FOLDER, DESCRIPTOR_SUBFOLDER)):
    if filename.endswith('.mat'):
        img_path = os.path.join(DESCRIPTOR_FOLDER, DESCRIPTOR_SUBFOLDER, filename)
        img_data = sio.loadmat(img_path)
        ALLFILES.append(img_path)
        ALLFEAT.append(img_data['F'][0])  # Assuming F is a 1D array
# Convert ALLFEAT to a numpy array
ALLFEAT = np.array(ALLFEAT)

# Pick a random image as the query
NIMG = ALLFEAT.shape[0]
queryimg = randint(0, NIMG - 1)

# Compute the distance between the query and all other descriptors
dst = []
query = ALLFEAT[queryimg]
for i in range(NIMG):
    candidate = ALLFEAT[i]
    distance = cvpr_compare(query, candidate)
    dst.append((distance, i))

# Sort the distances
dst.sort(key=lambda x: x[0])


In [101]:
# Show the top 15 results
SHOW = 15
for i in range(SHOW):
    print(ALLFILES[dst[i][1]])
    img_path = ALLFILES[dst[i][1]].replace('.mat', '.jpg')  # Assuming image files are .jpg
    img = cv2.imread(img_path)
    if img is not None:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB for matplotlib
        plt.figure(figsize=(10, 5))
        plt.imshow(img)
        plt.title(f"Result {i+1}")
        plt.axis('off')
        plt.show()
    else:
        print(f"Image not found: {img_path}")

AttributeError: 'NoneType' object has no attribute 'shape'